# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [3]:
import matplotlib.pyplot as plt
#import mplleaflet
import pandas as pd
import numpy as np


def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

#leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [4]:
%matplotlib notebook
# obtain the data over year 2005 - 2014
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df1 = df[(df['Date']<= '2014-12-31') & (df['Date']>= '2005-01-01')].copy()

# sort data by date
df1 = df1.sort_values('Date')

# remove the year, because we want to use groupby day
df1['Date'] = df1['Date'].str.split('-').str[1:].str.join('-')

# drop leap day 02-29
df2 = df1[df1['Date']!='02-29']

# group by date and obtain the min and max temperature over year 2005-2014
df3 = df2.groupby('Date')['Data_Value'].agg({'min': np.min, 'max': np.max})
df3.reset_index(inplace = True)
df3.index += 1

# create date, max and min temp ndarry
date = np.array(list(df3.index))
Temp_max = np.array(df3['max'].values.tolist(), dtype = int)/10
Temp_min = np.array(df3['min'].values.tolist(), dtype = int)/10

# a line graph of the record high and record low temperatures by day of the year over the period 2005-2014
plt.figure(figsize=(8,6))
plt.plot(date, Temp_max, 'cyan', label = 'Maximum for the day between 2005-2014', alpha=0.5)
plt.plot(date, Temp_min, 'lightsalmon', label = 'Minimum for the day between 2005-2014', alpha=0.5)

plt.gca().fill_between(range(len(date)), 
                       Temp_min, Temp_max, 
                       facecolor='gainsboro')

# calculate the break recond points in 2015
df_2015 = df[(df['Date']<= '2015-12-31') & (df['Date']>= '2015-01-01')].copy()
df_2015_1 = df_2015.groupby('Date')['Data_Value'].agg({'min': np.min, 'max': np.max})
df_2015_1.sort_index(inplace = True)
df_2015_1 = df_2015_1.reset_index()
df_2015_1.index += 1 
df_min = df_2015_1[df_2015_1['min'] < df3['min']]
df_max = df_2015_1[df_2015_1['max'] > df3['max']]

# create the needed ndarry for scatter plot
date_min_break = np.array(list(df_min.index))
Temp_min_break = np.array(df_min['min'].values.tolist(), dtype = int)/10
date_max_break = np.array(list(df_max.index))
Temp_max_break = np.array(df_max['max'].values.tolist(), dtype = int)/10

# plot scatter points
plt.scatter(date_min_break, Temp_min_break, s=10, c='darkblue', label='Colder day in 2015 than the record for the day between 2005-2014')
# plot a second data series 'Short students' in blue using the last three elements of x and y 
plt.scatter(date_max_break, Temp_max_break, s=10, c='darkred', label='Hotter day in 2015 than the record for the day between 2005-2014')

# add lable, legend, title and make plot beautiful
plt.xlabel('Day of the year')
plt.ylabel('Temperature(Celcius)')
plt.title('Min and Max Temperature Comparison: 2005-2014 vs 2015')
ax = plt.gca()
ax.axis([0,400,-40,80])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xticks(np.arange(0, 440, 40))
plt.legend(loc = 'upper center', frameon = False)
plt.savefig('assignment2.png')

FileNotFoundError: [Errno 2] File data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv does not exist: 'data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv'